In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L2\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['COAST.csv', 'EAST.csv', 'FWEST.csv', 'NCENT.csv', 'NORTH.csv', 'SCENT.csv', 'SOUTH.csv', 'WEST.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[6]))

,Net,Temprature
0,3607.904503,5.8
1,3609.619438,5.7
2,3573.158363,5.8
3,3538.420320,5.7
4,3542.905394,5.4
...,...,...
43818,3181.359858,18.1
43819,3044.648834,17.8
43820,2930.563230,17.2
43821,2829.531899,16.6


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[6]

zoneData = pd.read_csv(os.path.join(path, fileName))
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_SOUTH,Temp_SOUTH
0,3607.904503,5.8
1,3607.904503,5.8
2,3609.619438,5.7
3,3573.158363,5.8
4,3538.420320,5.7
...,...,...
43819,3181.359858,18.1
43820,3044.648834,17.8
43821,2930.563230,17.2
43822,2829.531899,16.6


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_SOUTH,Temp_SOUTH,Lag-2,Lag-7
0,3487.058100,15.9,3588.640200,3607.904503
1,3425.426721,14.8,3416.890946,3607.904503
2,3434.221408,12.5,3328.997595,3609.619438
3,3482.411848,10.9,3310.251773,3573.158363
4,3576.147399,10.4,3326.862273,3538.420320
...,...,...,...,...
43651,3181.359858,18.1,3003.465976,3033.234778
43652,3044.648834,17.8,2959.195602,2913.452696
43653,2930.563230,17.2,2905.932666,2828.170796
43654,2829.531899,16.6,2825.076669,2778.351469


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 14s 9ms/step - loss: 0.0125 - mape: 4545.5591 - mae: 0.0838 - val_loss: 0.0050 - val_mape: 18.5479 - val_mae: 0.0551
Epoch 2/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0056 - mape: 3544.0728 - mae: 0.0569 - val_loss: 0.0366 - val_mape: 36.0426 - val_mae: 0.1459
Epoch 3/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0044 - mape: 2943.9072 - mae: 0.0501 - val_loss: 0.0284 - val_mape: 36.2436 - val_mae: 0.1312
Epoch 4/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0040 - mape: 2878.8950 - mae: 0.0482 - val_loss: 0.0248 - val_mape: 32.7874 - val_mae: 0.1249
Epoch 5/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0037 - mape: 2704.4062 - mae: 0.0464 - val_loss: 0.0353 - val_mape: 36.6910 - val_mae: 0.1428
Epoch 6/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0035 - mape: 2017.4102 - mae: 0.0452 - val_loss: 0.0201

1105/1105 [==============================] - 8s 7ms/step - loss: 0.0016 - mape: 715.3013 - mae: 0.0306 - val_loss: 0.0258 - val_mape: 32.3734 - val_mae: 0.1256
Epoch 49/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0016 - mape: 824.0928 - mae: 0.0303 - val_loss: 0.0245 - val_mape: 31.7466 - val_mae: 0.1227
Epoch 50/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0015 - mape: 815.9623 - mae: 0.0300 - val_loss: 0.0252 - val_mape: 35.5632 - val_mae: 0.1313
Epoch 51/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0016 - mape: 634.2921 - mae: 0.0303 - val_loss: 0.0257 - val_mape: 32.8041 - val_mae: 0.1263
Epoch 52/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0015 - mape: 784.7373 - mae: 0.0295 - val_loss: 0.0226 - val_mape: 33.0183 - val_mae: 0.1222
Epoch 53/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0015 - mape: 821.8857 - mae: 0.0296 - val_loss: 0.0295 - val_mape: 3

1105/1105 [==============================] - 8s 7ms/step - loss: 9.9837e-04 - mape: 689.6562 - mae: 0.0241 - val_loss: 0.0265 - val_mape: 34.4780 - val_mae: 0.1306
Epoch 96/100
1105/1105 [==============================] - 8s 7ms/step - loss: 9.8127e-04 - mape: 745.5559 - mae: 0.0239 - val_loss: 0.0286 - val_mape: 34.3994 - val_mae: 0.1330
Epoch 97/100
1105/1105 [==============================] - 8s 7ms/step - loss: 9.3966e-04 - mape: 759.8892 - mae: 0.0235 - val_loss: 0.0250 - val_mape: 33.6029 - val_mae: 0.1271
Epoch 98/100
1105/1105 [==============================] - 8s 7ms/step - loss: 9.4996e-04 - mape: 787.1582 - mae: 0.0235 - val_loss: 0.0281 - val_mape: 34.4995 - val_mae: 0.1325
Epoch 99/100
1105/1105 [==============================] - 8s 7ms/step - loss: 9.2852e-04 - mape: 686.2654 - mae: 0.0233 - val_loss: 0.0260 - val_mape: 34.0167 - val_mae: 0.1294
Epoch 100/100
1105/1105 [==============================] - 8s 7ms/step - loss: 9.5768e-04 - mape: 820.0742 - mae: 0.0236 - val_l

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 842.3956274986267


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 9ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.17003721, 0.13916573, 0.11989564, ..., 0.247644  , 0.20992139,
       0.18223713], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.39291765, 0.359126  , 0.33636044, ..., 0.24270501, 0.21913098,
       0.200035  ])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,3574.330342,2619.130615
1,3429.509300,2486.824219
2,3331.942799,2404.238525
3,3289.798182,2387.514648
4,3339.593959,2430.484131
...,...,...
4363,3181.359858,3150.912354
4364,3044.648834,3091.416748
4365,2930.563230,2951.730469
4366,2829.531899,2790.062256


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

13.825613382234977